In [11]:
import pandas as pd

# Load the DataFrame from the pickle file
df = pd.read_pickle('prj7_presov_2.pkl')

In [15]:
df.iloc[0]

_id                                      64cbb610d0c120579c9da338
meta            {'power': {'battery': {'V': 6.665999889}, 'sol...
time            {'server': {'epoch': 1691072016.0666218, 'UTC'...
measurements                                                  NaN
tare_vals                                                     NaN
Name: 0, dtype: object

In [16]:
# Function to convert 'UTC' timestamp to datetime
def convert_utc_to_datetime(row):
    try:
        return pd.to_datetime(row['time']['server']['UTC'])
    except (KeyError, TypeError):
        return pd.NaT  # Return NaT (Not-a-Time) for missing or incorrect data


# Apply the conversion function to create a new column 'time_server_UTC'
df['time_server_UTC'] = df.apply(convert_utc_to_datetime, axis=1)

In [17]:
import plotly.express as px

# Create a histogram using Plotly Express
fig = px.histogram(df, x='time_server_UTC', title='Timestamp Histogram')
fig.update_xaxes(title_text='Timestamp')
fig.update_yaxes(title_text='Count')

# Show the plot
fig.show()

In [18]:
df = df[df['time_server_UTC'] >= '2023-10-21']

In [ ]:
M022_keys = ['7', '8', '9', '13', '14', '15', '16', '19', '21', '28']
M023_keys = ['4', '5', '6', '10', '11', '12', '17', '18', '20']

In [43]:
import pandas as pd

# Assuming df is your original DataFrame

# Define the key sets for M022 and M023
M022_keys = set(['7', '8', '9', '13', '14', '15', '16', '19', '21', '28'])
M023_keys = set(['4', '5', '6', '10', '11', '12', '17', '18', '20'])

# Initialize lists to hold indices for M022 and M023
indices_M022 = []
indices_M023 = []

# Iterate over each row in the original DataFrame
for index, row in df.iterrows():
    if isinstance(row["measurements"], dict):  # Check if the value is a dictionary
        measurement_keys = set(row["measurements"].keys())

        # Check if any key in the row's measurements is in M022_keys
        if not measurement_keys.isdisjoint(M022_keys):
            indices_M022.append(index)

        # Similarly, for M023_keys
        if not measurement_keys.isdisjoint(M023_keys):
            indices_M023.append(index)
    else:
        # Handle rows where "measurements" is not a dictionary (e.g., NaN or other types)
        # You might want to append these to a separate list for review or handle them as needed
        pass

# Use the indices to select rows for each DataFrame
df_M022 = df.loc[indices_M022].reset_index(drop=True)
df_M023 = df.loc[indices_M023].reset_index(drop=True)

# df_M022 and df_M023 now contain the separated rows based on the specified keys

In [47]:
def flatten_measurements(df):
    # Initialize a list to accumulate the new rows
    new_rows = []

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Extract the row (excluding 'measurements') as a dictionary
        new_row = row.drop('measurements').to_dict()

        # Flatten 'measurements' if it is a dictionary
        if isinstance(row['measurements'], dict):
            for key, values in row['measurements'].items():
                for sub_key, value in values.items():
                    # Create a new column name for each sub_key in the measurements
                    new_col_name = f'{key}.{sub_key}'
                    new_row[new_col_name] = value

        # Append the new row dictionary to the list
        new_rows.append(new_row)

    # Create a DataFrame from the list of new row dictionaries
    flattened_df = pd.DataFrame(new_rows)

    return flattened_df


# Apply the function to both DataFrames
df_M022_flattened = flatten_measurements(df_M022)
df_M023_flattened = flatten_measurements(df_M023)

In [55]:
def drop_columns_based_on_rules(df):
    cols_to_drop = []
    for col in df.columns:
        # Rule for columns starting with "20" or "21"
        if col.startswith('20') or col.startswith('21'):
            if col.endswith('pv0') or col.endswith('pv3'):
                cols_to_drop.append(col)
        # Rule for other numbered columns
        elif col[0].isdigit() and not col.startswith('20') and not col.startswith('21'):
            if col.endswith('pv2') or col.endswith('pv3'):
                cols_to_drop.append(col)
    # Drop the selected columns
    df.drop(columns=cols_to_drop, inplace=True)


# Apply the function to both flattened DataFrames
drop_columns_based_on_rules(df_M022_flattened)
drop_columns_based_on_rules(df_M023_flattened)

In [58]:
# Columns to be dropped
cols_to_drop = ['_id', 'meta', 'time', 'tare_vals']

# Drop the specified columns from df_M022_flattened if they exist
df_M022_flattened = df_M022_flattened.drop(
    columns=cols_to_drop, errors='ignore')

# Drop the specified columns from df_M023_flattened if they exist
df_M023_flattened = df_M023_flattened.drop(
    columns=cols_to_drop, errors='ignore')

In [59]:
df_M022_flattened.columns

Index(['time_server_UTC', '7.pv0', '7.pv1', '8.pv0', '8.pv1', '9.pv0', '9.pv1',
       '13.pv0', '13.pv1', '14.pv0', '14.pv1', '15.pv0', '15.pv1', '16.pv0',
       '16.pv1', '19.pv0', '19.pv1', '21.pv1', '21.pv2', '28.pv0', '28.pv1'],
      dtype='object')

In [60]:
df_M023_flattened.columns

Index(['time_server_UTC', '4.pv0', '4.pv1', '5.pv0', '5.pv1', '6.pv0', '6.pv1',
       '10.pv0', '10.pv1', '11.pv0', '11.pv1', '12.pv0', '12.pv1', '17.pv0',
       '17.pv1', '18.pv0', '18.pv1', '20.pv1', '20.pv2'],
      dtype='object')

In [71]:
df_M022_flattened.to_pickle("df_M022_flattened.pkl")
df_M023_flattened.to_pickle("df_M023_flattened.pkl")

In [70]:
df_M023_flattened

,time_server_UTC,4.pv0,4.pv1,5.pv0,5.pv1,6.pv0,6.pv1,10.pv0,10.pv1,11.pv0,11.pv1,12.pv0,12.pv1,17.pv0,17.pv1,18.pv0,18.pv1,20.pv1,20.pv2
0,2023-10-21 00:07:50,0.400275,0.403664,0.613413,0.583559,0.390089,0.633985,0.457571,0.501602,0.280812,0.328344,0.419798,0.270224,0.595622,0.497091,0.290948,0.545957,15.1875,16.3750
1,2023-10-21 00:17:50,0.400309,0.403695,0.613414,0.583564,0.390061,0.633939,0.457564,0.501622,0.280793,0.328329,0.419761,0.270272,0.595656,0.497105,0.291065,0.545940,15.1875,16.3750
2,2023-10-21 00:27:43,0.400224,0.403650,0.613440,0.583581,0.390071,0.633962,0.457602,0.501632,0.280788,0.328326,0.419780,0.270236,0.595667,0.497098,0.291096,0.546000,15.2500,16.3750
3,2023-10-21 00:37:39,0.400238,0.403685,0.613476,0.583514,0.390086,0.633994,0.457579,0.501647,0.280799,0.328332,0.419848,0.270290,0.595739,0.497096,0.291309,0.545996,15.2500,16.3750
4,2023-10-21 00:47:37,0.400260,0.403702,0.613453,0.583540,0.390061,0.633958,0.457640,0.501619,0.280784,0.328320,0.419687,0.270237,0.595680,0.497102,0.291274,0.545968,15.2500,16.3750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8941,2024-02-05 12:02:09,0.393505,0.396647,0.611041,0.579851,0.385521,0.633311,0.459737,0.504798,0.278438,0.327910,0.418551,0.269290,0.585850,0.495744,0.283209,0.549402,6.0625,7.2500
8942,2024-02-05 12:12:06,0.393545,0.396710,0.611030,0.579894,0.385541,0.633321,0.459765,0.504772,0.278458,0.327895,0.418537,0.269288,0.585847,0.495753,0.283023,0.549473,6.1875,7.3750
8943,2024-02-05 12:22:03,0.393634,0.396766,0.611015,0.579852,0.385594,0.633373,0.459731,0.504740,0.278413,0.327915,0.418533,0.269295,0.585781,0.495729,0.282833,0.549454,6.2500,7.5000
8944,2024-02-05 12:32:00,0.393626,0.396795,0.611082,0.579884,0.385492,0.633350,0.459770,0.504771,0.278387,0.327864,0.418546,0.269304,0.585897,0.495741,0.283323,0.549453,6.3125,7.6250


In [64]:
df

,_id,meta,time,measurements,tare_vals,time_server_UTC
19358,6533165638f7d3b3223ec3d3,"{'power': {'battery': {'V': 7.897999763}, 'sol...","{'server': {'epoch': 1697846870.7420585, 'UTC'...","{'4': {'pv0': 0.400274515, 'pv1': 0.403664231,...",NaN,2023-10-21 00:07:50
19359,653318ae38f7d3b3223ec3d5,"{'power': {'battery': {'V': 7.864999771}, 'sol...","{'server': {'epoch': 1697847470.111433, 'UTC':...","{'4': {'pv0': 0.400309443, 'pv1': 0.403695375,...",NaN,2023-10-21 00:17:50
19360,65331aff38f7d3b3223ec3d7,"{'power': {'battery': {'V': 7.854000092}, 'sol...","{'server': {'epoch': 1697848063.671707, 'UTC':...","{'4': {'pv0': 0.400224447, 'pv1': 0.403650373,...",NaN,2023-10-21 00:27:43
19361,65331d5338f7d3b3223ec3d9,"{'power': {'battery': {'V': 7.831999779}, 'sol...","{'server': {'epoch': 1697848659.522527, 'UTC':...","{'4': {'pv0': 0.400237888, 'pv1': 0.403684855,...",NaN,2023-10-21 00:37:39
19362,65331fa938f7d3b3223ec3db,"{'power': {'battery': {'V': 7.875999928}, 'sol...","{'server': {'epoch': 1697849257.1925848, 'UTC'...","{'4': {'pv0': 0.400259554, 'pv1': 0.403701782,...",NaN,2023-10-21 00:47:37
...,...,...,...,...,...,...
42677,65c0d54038f7d3b322404fa2,"{'power': {'battery': {'V': 6.116000175}, 'sol...","{'server': {'epoch': 1707136320.1542385, 'UTC'...","{'4': {'pv0': 0.393626243, 'pv1': 0.396795362,...",NaN,2024-02-05 12:32:00
42678,65c0d56438f7d3b322404fa3,"{'power': {'battery': {'V': 4.159999847}, 'sol...","{'server': {'epoch': 1707136356.1958656, 'UTC'...","{'7': {'pv0': 0.437466443, 'pv1': 0.545305192,...",NaN,2024-02-05 12:32:36
42679,65c0d79538f7d3b322404faa,"{'power': {'battery': {'V': 6.19299984}, 'sola...","{'server': {'epoch': 1707136917.710205, 'UTC':...","{'4': {'pv0': 0.393749237, 'pv1': 0.396874398,...",NaN,2024-02-05 12:41:57
42680,65c0d7b938f7d3b322404fab,"{'power': {'battery': {'V': 4.150000095}, 'sol...","{'server': {'epoch': 1707136953.4912567, 'UTC'...","{'7': {'pv0': 0.437489003, 'pv1': 0.545337081,...",NaN,2024-02-05 12:42:33


In [69]:
df_M023_flattened

,time_server_UTC,4.pv0,4.pv1,5.pv0,5.pv1,6.pv0,6.pv1,10.pv0,10.pv1,11.pv0,11.pv1,12.pv0,12.pv1,17.pv0,17.pv1,18.pv0,18.pv1,20.pv1,20.pv2
0,2023-10-21 00:07:50,0.400275,0.403664,0.613413,0.583559,0.390089,0.633985,0.457571,0.501602,0.280812,0.328344,0.419798,0.270224,0.595622,0.497091,0.290948,0.545957,15.1875,16.3750
1,2023-10-21 00:17:50,0.400309,0.403695,0.613414,0.583564,0.390061,0.633939,0.457564,0.501622,0.280793,0.328329,0.419761,0.270272,0.595656,0.497105,0.291065,0.545940,15.1875,16.3750
2,2023-10-21 00:27:43,0.400224,0.403650,0.613440,0.583581,0.390071,0.633962,0.457602,0.501632,0.280788,0.328326,0.419780,0.270236,0.595667,0.497098,0.291096,0.546000,15.2500,16.3750
3,2023-10-21 00:37:39,0.400238,0.403685,0.613476,0.583514,0.390086,0.633994,0.457579,0.501647,0.280799,0.328332,0.419848,0.270290,0.595739,0.497096,0.291309,0.545996,15.2500,16.3750
4,2023-10-21 00:47:37,0.400260,0.403702,0.613453,0.583540,0.390061,0.633958,0.457640,0.501619,0.280784,0.328320,0.419687,0.270237,0.595680,0.497102,0.291274,0.545968,15.2500,16.3750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8941,2024-02-05 12:02:09,0.393505,0.396647,0.611041,0.579851,0.385521,0.633311,0.459737,0.504798,0.278438,0.327910,0.418551,0.269290,0.585850,0.495744,0.283209,0.549402,6.0625,7.2500
8942,2024-02-05 12:12:06,0.393545,0.396710,0.611030,0.579894,0.385541,0.633321,0.459765,0.504772,0.278458,0.327895,0.418537,0.269288,0.585847,0.495753,0.283023,0.549473,6.1875,7.3750
8943,2024-02-05 12:22:03,0.393634,0.396766,0.611015,0.579852,0.385594,0.633373,0.459731,0.504740,0.278413,0.327915,0.418533,0.269295,0.585781,0.495729,0.282833,0.549454,6.2500,7.5000
8944,2024-02-05 12:32:00,0.393626,0.396795,0.611082,0.579884,0.385492,0.633350,0.459770,0.504771,0.278387,0.327864,0.418546,0.269304,0.585897,0.495741,0.283323,0.549453,6.3125,7.6250
